# 03 – Data Cleaning (Library Borrowings)

**Purpose:** Create an analysis-ready dataset by applying a small set of clearly defined cleaning rules based on the sanity-check findings.

This notebook:
- loads the merged dataset,
- parses key columns (timestamps, numeric fields),
- removes records with critical inconsistencies
- saves a cleaned dataset for EDA.

**Not included:**
- no exploratory analysis or visualizations beyond basic before/after counts


In [19]:
%config InlineBackend.figure_format = 'retina'

from pathlib import Path
import numpy as np
import pandas as pd

from utils import setup_pandas, setup_plotting


# --- global notebook setup (pandas + tueplots/matplotlib style) ---
setup_pandas()
setup_plotting()


# --- project paths ---
PROCESSED_DATA_PATH = Path('../dat/processed')
RAW_DATA_PATH = Path('../dat/raw')

DATA_FILE = PROCESSED_DATA_PATH / "borrowings_2019_2025.csv"
CLEANED_FILE = PROCESSED_DATA_PATH / "borrowings_2019_2025_cleaned.csv"

CLOSED_DAYS_FILE = RAW_DATA_PATH / "closed_days_2019_2025.csv"

In [20]:
# --- load data ---
borrowings = pd.read_csv(
    DATA_FILE,
    sep=";",
    quotechar='"',
    encoding="utf-8"
)

closed_days = pd.read_csv(
    CLOSED_DAYS_FILE,
    sep=";",
    quotechar='"',
    encoding="utf-8"
)

print("Loaded shape:", borrowings.shape)

Loaded shape: (2407610, 17)


In [21]:
# --- preprocess relevant columns ---

# column names
ISSUE_COL = "Ausleihdatum/Uhrzeit"
RETURN_COL = "Rückgabedatum/Uhrzeit"

DURATION_COL = "Leihdauer"
EXT_COL = "Anzahl_Verlängerungen"
LATE_FLAG_COL = "Verspätet"
LATE_DAYS_COL = "Tage_zu_spät"

ID_COL = "issue_id"
USER_CATEGORY_COL = "Benutzerkategorie"
USER_COL = "Benutzer-Systemnummer"
BARCODE_COL = "Barcode"

MEDIA_TYPE_COL = "Medientyp"
CCODE_COL = "Sammlungszeichen/CCODE"

# timestamps
for c in [ISSUE_COL, RETURN_COL]:
    if c in borrowings.columns:
        borrowings[c] = pd.to_datetime(borrowings[c], errors="coerce")

# numeric columns
for c in [DURATION_COL, EXT_COL, LATE_DAYS_COL]:
    if c in borrowings.columns:
        borrowings[c] = pd.to_numeric(borrowings[c], errors="coerce")

# normalize late flag to boolean (Ja/Nein -> True/False); keep unknown as <NA>
if LATE_FLAG_COL in borrowings.columns:
    v = borrowings[LATE_FLAG_COL].astype(str).str.strip().str.lower()
    borrowings["late_bool"] = pd.Series(np.where(v == "ja", True, np.where(v == "nein", False, pd.NA)), dtype="boolean")

## Cleaning rules

We remove records that violate fundamental consistency constraints of borrowing transactions:

1. Missing return and leihdauer timestamp
2. Leihdauer exceeds the max durration given by the 28 days and a max of 6 extensions.
3. Non-representative users are removed
4. Removal of media types (optional)
5. Removal of ccodes (optional)

**ATTENTION: So far, only rules for the loan period and return have been formulated.
Further rules should be defined for further analyses on other columns!**

No other filtering is applied at this stage.

### 1. Missing return timestamp and leihdauer timestamp

Records without a return timestamp, corresponding to items not yet returned at the time of data extraction, were excluded from further analysis.

In [22]:
# --- heck assumption: missing return timestamp ⇔ missing duration ---
mask_return_missing = borrowings[RETURN_COL].isna()
mask_duration_missing = borrowings[DURATION_COL].isna()

check_table = pd.crosstab(
    mask_return_missing,
    mask_duration_missing,
    rownames=["return_missing"],
    colnames=["duration_missing"]
)

display(check_table)

# verify assumption
if check_table.loc[True, False] == 0:
    print("Assumption holds: whenever return timestamp is missing, duration is also missing.")
else:
    print("WARNING: There are cases with missing return timestamp but present duration.")

# remove not-yet-returned items (missing return timestamp)
before_n = len(borrowings)
borrowings = borrowings.loc[~mask_return_missing].copy()
after_n = len(borrowings)

print(f"Removed {before_n - after_n} rows with missing return timestamp.")
print("Remaining rows:", after_n)


duration_missing,False,True
return_missing,,
False,2358824,0
True,0,48786


Assumption holds: whenever return timestamp is missing, duration is also missing.
Removed 48786 rows with missing return timestamp.
Remaining rows: 2358824


### 2. removal of loans exceeding the maximum allowed duration

Loan durations are recalculated in terms of **library open days**, accounting for weekly closing days (Sunday and Monday) and official closure days.
Based on the borrowing rules, a maximum loan duration is derived as 28 days plus allowed extensions (capped at six).

Loans whose open-day duration exceeds this rule-based maximum are flagged as implausible and removed.
This step ensures that the dataset reflects valid borrowing behavior and excludes records likely caused by data errors or exceptional administrative cases.


In [23]:
# --- remove loans exceeding max loan duration ---
closed_days["schliesstag"] = pd.to_datetime(closed_days["schliesstag"], dayfirst=True, errors="coerce")
holidays = (
    closed_days["schliesstag"]
    .dropna()
    .dt.normalize()
    .drop_duplicates()
    .values.astype("datetime64[D]")
)

start = borrowings[ISSUE_COL].dt.normalize().values.astype("datetime64[D]")
end = borrowings[RETURN_COL].dt.normalize().values.astype("datetime64[D]")

# library open days: Tue–Sat (closed every Sunday+Monday) => Mon..Sun: 0,1,1,1,1,1,0
weekmask = "0111110"

open_days = np.full(len(borrowings), np.nan, dtype="float64")
valid = (~np.isnan(start) & ~np.isnan(end)) & (end >= start)

open_days[valid] = np.busday_count(
    start[valid],
    end[valid],     # counts in [start, end)
    weekmask=weekmask,
    holidays=holidays
).astype("float64")

borrowings["open_days_leihdauer"] = open_days

#  allowed max = 28 days * (1 + extensions), with extensions capped at 6
ext = pd.to_numeric(borrowings[EXT_COL], errors="coerce").fillna(0).clip(lower=0, upper=6)
borrowings["max_allowed_open_days"] = 28 * (1 + ext)

borrowings["weird_loan"] = (borrowings["open_days_leihdauer"] > borrowings["max_allowed_open_days"]).fillna(False)

borrowings.loc[borrowings["weird_loan"]].sort_values(
    "open_days_leihdauer", ascending=False
)[
    [ISSUE_COL, DURATION_COL, "open_days_leihdauer", EXT_COL, "max_allowed_open_days", "weird_loan"]
].head(10)


# removing loans exceeding allowed open days
before_n = len(borrowings)
borrowings = borrowings.loc[~borrowings["weird_loan"]].copy()

after_n = len(borrowings)
removed_n = before_n - after_n

print(f"Removed {removed_n} rows with Leihdauer exceeding max loan duration.")
print("Remaining rows:", after_n)


Removed 8625 rows with Leihdauer exceeding max loan duration.
Remaining rows: 2350199


### 3. removal of system users

System and administrative accounts (`SYS`, `MDA`) are excluded, as they do not represent regular user behavior.
Removing these records prevents technical or administrative activity from biasing the analysis.

In [24]:
# --- remove non-representative users (system / administrative accounts) ---
before_n = len(borrowings)

borrowings = borrowings.loc[
    ~borrowings[USER_CATEGORY_COL].isin(["SYS", "MDA"])
].copy()

after_n = len(borrowings)

print(f"Removed {before_n - after_n} rows from system/administrative users (SYS, MDA).")
print("Remaining rows:", after_n)


Removed 37127 rows from system/administrative users (SYS, MDA).
Remaining rows: 2313072


### 4. Removal of media types (optional)

For some analyses, it can be useful to remove rarely used media types. You can also configure how media types are classified into book and non-book categories here. By default, no media types are removed. Update the `BOOK_TYPES_ONLY` flag or the variables `UNINTERESTING_MEDIA_TYPES` and `MIN_BORROWINGS_PER_MEDIA_TYPE` to change this behavior.

In [25]:
BOOK_TYPES_ONLY = False
UNINTERESTING_MEDIA_TYPES = [
    #"Sonstiges" # mixed media type with unknown contents
]
MIN_BORROWINGS_PER_MEDIA_TYPE = 0

book_categories = [
    "Sachbuch",
    "Belletristik",
    "Fremdsprachige Belletristik",
    "Kinder u. Jugendbuch",
    "Comic",
    "Zeitschriften",
]

# only book types
if BOOK_TYPES_ONLY:
    borrowings = borrowings[borrowings[MEDIA_TYPE_COL].isin(book_categories)].reset_index(drop=True)

# apply min borrowings filter
counts = borrowings[MEDIA_TYPE_COL].value_counts()
keep_types = counts.index[counts >= MIN_BORROWINGS_PER_MEDIA_TYPE]
borrowings = borrowings[borrowings[MEDIA_TYPE_COL].isin(keep_types)].reset_index(drop=True)

# remove uninteresting types
borrowings = borrowings[~borrowings[MEDIA_TYPE_COL].isin(UNINTERESTING_MEDIA_TYPES)].reset_index(drop=True)


### 5. Removal of CCodes (optional)

For some analyses, it can be useful to remove rarely used CCodes. By default, no ccodes are removed. Update the variable `MIN_BORROWINGS_PER_CCODE` to change this behavior.

In [26]:
MIN_BORROWINGS_PER_CCODE = 100

# apply min borrowings filter
counts = borrowings[CCODE_COL].value_counts()
keep_types = counts.index[counts >= MIN_BORROWINGS_PER_CCODE]
borrowings = borrowings[borrowings[CCODE_COL].isin(keep_types)].reset_index(drop=True)

## Save cleaned Dataset

In [27]:
borrowings.to_csv(CLEANED_FILE, index=False, sep=";", quotechar='"', encoding="utf-8")
print("Saved cleaned dataset to:", CLEANED_FILE)

Saved cleaned dataset to: ../dat/processed/borrowings_2019_2025_cleaned.csv


## Cleaning summary

- missing return stamp and leihdauer were removed, also loan period longer than 365 days
- The resulting dataset is intended as the input for EDA and modeling notebooks.

